In [9]:
!pip install pyserial

In [65]:
import serial

START_TEST        = 0b100
BEFORE_SOLENOID   = 0b001
AFTER_SOLENOID    = 0b010
BEAM_READ         = 0b011 
END_TEST          = 0b110   

collected_data = []

def read_data(ser):
    stop = False
    while not stop:
        if ser.in_waiting >= 7:
            data = ser.read(7)
        
            identifier = (data[0] >> 5) & 0x07
            if identifier == END_TEST:
                stop = True
            
            delimiter = data[6] & 0xFF
                
            if delimiter == 0xFF:
                collected_data.append(data)
               # print(f"Identifier: {identifier}, Analog Value: {analog_value}, Time: {time_micros}")
            else:
                print("Delimiter not found, packet discarded.")

if __name__ == "__main__":
    ser = serial.Serial('/dev/cu.usbserial-110', 9600)  # Adjust the port as needed
    read_data(ser)


Identifier: 4, Analog Value: None, Time: 65343055
Identifier: 1, Analog Value: None, Time: 65343065
Identifier: 2, Analog Value: None, Time: 65443003
Identifier: 3, Analog Value: 41, Time: 65443098
Identifier: 3, Analog Value: 41, Time: 65443189
Identifier: 3, Analog Value: 40, Time: 65443279
Identifier: 3, Analog Value: 41, Time: 65443369
Identifier: 3, Analog Value: 41, Time: 65443459
Identifier: 3, Analog Value: 41, Time: 65443554
Identifier: 3, Analog Value: 41, Time: 65443644
Identifier: 3, Analog Value: 41, Time: 65443734
Identifier: 3, Analog Value: 41, Time: 65443824
Identifier: 3, Analog Value: 41, Time: 65450486
Identifier: 3, Analog Value: 41, Time: 65480798
Identifier: 3, Analog Value: 41, Time: 65480891
Identifier: 3, Analog Value: 41, Time: 65480981
Identifier: 3, Analog Value: 41, Time: 65481072
Identifier: 3, Analog Value: 41, Time: 65516321
Identifier: 3, Analog Value: 41, Time: 65516414
Identifier: 3, Analog Value: 42, Time: 65516504
Identifier: 3, Analog Value: 41, T

In [ ]:
identifier = (data[0] >> 5) & 0x07

if identifier == BEAM_READ:
    analog_value = ((data[0] & 0x1F) << 5) | ((data[1] >> 3) & 0x1F)
elif identifier == END_TEST:
    stop = True
else:
    analog_value = None
    time_micros = (data[2] << 24) | (data[3] << 16) | (data[4] << 8) | data[5]